In [1]:
%matplotlib qt
import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy import stats
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import gc
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.externals import joblib
from sklearn.svm import SVC
import sklearn.metrics as metrics
import xgboost as xgb
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

path = '../../data/Debt issuing company 2018 report/'

In [2]:
# 去掉异常值

def drop_out(frame,col,model='Confidence interval',t_alpha=0.95,alpha=2,IQR_rate=1.5,quantile=0.95):
    '''modle: 'gauss','box','quantile' '''
    
    if model == 'Confidence interval':
        u_ = frame[col].mean()
        v_ = frame[col].std()
        interval_ = stats.t.interval(t_alpha,frame[col].count()-1,u_,v_)
        cond_ = (frame[col]<interval_[1])&(frame[col]>interval_[0])
    
    elif model == 'gauss':
        u_ = frame[col].mean()
        v_ = frame[col].std()
        cond_ = (frame[col]-u_)/v_ < alpha
    
    elif model == 'box':
        q1 = frame[col].quantile(0.25)
        q3 = frame[col].quantile(0.75)
        IQR = (q3-q1)*IQR_rate
        q1 -= IQR ; q3 += IQR
        cond_ = (frame[col]<q3)&(frame[col]>q1)
    
    elif model == 'quantile':
        top_ = frame[col].quantile(quantile)
        bottom_ = frame[col].quantile(1-quantile)
        cond_ = (frame[col]<top_)&(frame[col]>bottom_)
    
    else:
        print('please try again')
        return frame
    
    index_ = np.where(frame[col]!=frame[col],True,
                                                  np.where(cond_,True,False))
    frame = frame.loc[index_,:]
    return frame

# 时间操作

def mms(frame,groupby,col,):
    max_ = dict(frame.groupby(by=groupby,)[col].max())
    min_ = dict(frame.groupby(by=groupby,)[col].min())
    max_ = frame[groupby].map(lambda x:max_[x])
    min_ = frame[groupby].map(lambda x:min_[x])
    result = (frame[col].values - min_)/(max_-min_)
    return [round(i,2) for i in result]

def moving(frame,groupby,time_col,col,wins=3,weight=[0.55,0.3,0.15]):
    '''wins是窗口数,weight是权重,这两个必须对应'''
    frame = frame.sort_values([groupby,time_col])
    for i in range(wins):
        if i == 0 :
            list_ = weight[i]*(frame[col].values)
        else :
            list_ += weight[i]*(frame.groupby(groupby)[col].rolling(i+1).mean().values)
    return list_

def bodonglv(frame,groupby,time_col,col,wins):
    frame = frame.sort_values([groupby,time_col])
    std_ = (frame.groupby(groupby)[col].rolling(wins).std().values)
    mean_ = (frame.groupby(groupby)[col].rolling(wins).mean().values)
    return [round(std_[i]/mean_[i],2) for i in range(len(std_))]

def tongbi(frame,groupby,time_col,col,diff):
    frame = frame.sort_values([groupby,time_col])
    diff_ = frame.groupby(groupby)[col].diff(diff).values
    ori_ = frame[col].values
    return [np.nan]*diff+[round(diff_[i]/abs(ori_[i-diff]),2) for i in range(diff,frame.shape[0])]

# 补充空白值

def diy_ss(frame,weight_dict,quantile=0.2):
    for i in frame.columns:
        if ('%' not in i) and ('/' not in i) and ('率' not in i) and ('倍数' not in i):
            if frame[i].min() >= 0:
                frame[i] = np.log1p(frame[i])
                weight_dict[i] = 'log1p'
            else :
                mean_ = frame[i][(frame[i]>=frame[i].quantile(quantile)) & (frame[i] <= frame[i].quantile(1-quantile))].mean()
                std_ = frame[i][(frame[i]>=frame[i].quantile(quantile)) & (frame[i] <= frame[i].quantile(1-quantile))].std()
                weight_dict[i] = [mean_,std_]
                frame[i] = (frame[i]-mean_)/std_
    return frame,weight_dict

def fillna_(frame, group_col, model_name = 'rf', show_rand = 0.25, n_epoch = 3):
    
    def return_index(aa,bb):
        j=0
        cc = []
        for i in range(len(aa)):
            if aa[i] == False :
                cc.append(aa[i])
            else:
                cc.append(bb[j])
                j += 1
        return cc
    
    frame_col = frame.columns
    
    for l,comp in enumerate(set(group_col)):

        index_y = list(group_col == comp)
        full_col = []
        loss_col = {}

        for col in frame.columns:
            if frame.loc[index_y,col].isnull().sum() == 0:
                full_col.append(col)
            else:
                loss_col[col] = frame.loc[index_y,col].isnull().sum()

        loss_col = sorted(loss_col.items(),key=lambda x:x[1])
        loss_col = [i[0] for i in loss_col]
        
        index_dict = {}
        if len(full_col) == 0:
            index_dict[loss_col[0]] = frame.loc[index_y,loss_col[0]].isnull()
            index_dict[loss_col[0]].fillna(index_dict[loss_col[0]].median(),inplace=True)
            full_col.append(loss_col[0])
            loss_col = loss_col[1:]
            
        for epoch in range(n_epoch):

            if epoch == 0:
                for _,col in enumerate(loss_col):
                    if np.random.rand()>(1-show_rand):
                        print(comp,f'{l}/{len(set(group_col))}',col,f'{_}/{len(loss_col)}')
                    index_l = list(frame.loc[index_y,col].isnull())
                    index_f = list(frame.loc[index_y,col].notnull())
                    index_l_ = return_index(index_y,index_l)
                    index_f_ = return_index(index_y,index_f)
                    index_dict[col] = (index_l_,index_f_)
                    if model_name == 'rf':
                        model = RandomForestRegressor(max_depth=4, max_features=0.9, n_jobs=-1)
                    else:
                        model = Pipeline([('ploy', PolynomialFeatures(),
                                     'lr', LogisticRegression(fit_intercept=False))])
                    model.fit(frame.loc[index_f_,full_col],frame.loc[index_f_,col])
                    pre = model.predict(frame.loc[index_l_,full_col])
                    frame.loc[index_l_,col] = pre
                    full_col.append(col)
            
            else:
                for col in index_dict:
                    index_l_ = index_dict[col][0]
                    index_f_ = index_dict[col][1]
                    if model_name == 'rf':
                        model = RandomForestRegressor(max_depth=4, max_features=0.6, n_jobs=-1)
                    else:
                        model = Pipeline([('ploy', PolynomialFeatures(),
                                     'lr', LogisticRegression(fit_intercept=False))])
                    model.fit(frame.loc[index_f_,full_col],frame.loc[index_f_,col])
                    pre = model.predict(frame.loc[index_l_,full_col])
                    frame.loc[index_l_,col] = pre
                    
    gc.collect()
    return frame[frame_col]

# 独热编码

def one_hot_str(frame,col,replace=True):
    if replace:
        a_ = frame.pop(col)
    else:
        a_ = frame[col]
    a_.fillna('miss',inplace=True)
    a_ = pd.get_dummies(a_,prefix=a_.name)
    frame = pd.concat([frame,a_],axis=1)
    del a_
    return frame
def one_hot_int(frame,col,number):
    a_ = frame.pop(col)
    a_ = pd.qcut(a_,number)
    col_name_ = [a_.name+'_'+str(i+1) for i in range(number)]
    a_ = pd.get_dummies(a_,)
    a_.columns = col_name_
    frame = pd.concat([frame,a_],axis=1)
    del a_,col_name_
    return frame

# 截取时间节点

def getxyb(date,delta_y = 1,):
    '''得到当前日期的前几年最后一天,默认1年'''
    aaa = date - pd.tseries.offsets.DateOffset(years=1*delta_y)
    return aaa + pd.tseries.offsets.DateOffset(years=1,months=1 - aaa.month, days= - aaa.day)
def getxsb(date,delta_s = 1,):
    '''得到当前日期的前几个季度最后一天,默认1季度'''
    aaa = date - pd.tseries.offsets.DateOffset(months=3*delta_s)
    return aaa + pd.tseries.offsets.DateOffset(months=3 - ((aaa.month - 1) % 3), days=-aaa.day)
def getxhyb(date,delta_hy = 1,):
    '''得到当前日期的前几个半年最后一天,默认1半年'''
    aaa = date - pd.tseries.offsets.DateOffset(months=6*delta_hy)
    return aaa + pd.tseries.offsets.DateOffset(months=6 - ((aaa.month - 1) % 6), days=-aaa.day)

# 原始财务数据读取+分布图

In [3]:
start_year = 2013
end_year = 2017
data_a = DataFrame()
for i in range(start_year, end_year+1):
    if i % 3 ==0:
        print('is concating {} {}/{}'.format(i, i-start_year+1, end_year+1-start_year))
    path_a = path+'y/{}y.xlsx'.format(i)
    data_a_ = pd.read_excel(path_a)[:-2]
    data_a_.drop(['是否经过审计','审计意见'] , axis=1, inplace=True)
    data_a = pd.concat([data_a, data_a_])
del data_a_
gc.collect()
print('finish concat data_y')

print(np.array(list(data_a.isnull().sum(0))))

is concating 2013 1/5
is concating 2016 4/5
finish concat data_y
[   0    0  177 2209  225  418 2782 2782  620  831  741  357  705 4956
  568  799 1400 2798  795 4990 4965 1771 2615 3442  735 2788 2789 2727
 2214 6719 5097]


In [4]:
data_a_ = pd.read_excel(path+'a/2017a.xlsx')[:-2]
data_a_ = data_a_.loc[data_a_['名称'].map(lambda x: '上海华信' in x), :][0:1]
data_a_ = data_a_.loc[data_a_['报告期'].dt.month == 9, :]
data_a_.drop(['是否经过审计','审计意见'] , axis=1, inplace=True)

data_a_[['主营业务收入(亿元)', '主营业务利润(亿元)', 'EBITDA(亿元)', '净利润(亿元)', 'EBITDA/带息债务']] = \
data_a_[['主营业务收入(亿元)', '主营业务利润(亿元)', 'EBITDA(亿元)', '净利润(亿元)', 'EBITDA/带息债务']].apply(
lambda x: 12*x/data_a_['报告期'].dt.month)
data_a_[['经营性现金流/EBITDA',]] = data_a_[['经营性现金流/EBITDA',]].apply(lambda x: x*data_a_['报告期'].dt.month/12)
data_a_['报告期'] = pd.to_datetime('2017-12-31')
data_a = pd.concat([data_a, data_a_],)

data_a.index = range(data_a.shape[0])

del data_a_

# 公司信息读取

In [5]:
indestry = pd.read_excel(path+'comp_feature/产业类发债企业行业分类0827.xlsx', sheet_name='产业类企业')
indestry.insert(1, '是否交通', 0)
transport = pd.read_excel(path+'comp_feature/产业类发债企业行业分类0827.xlsx', sheet_name='交通运输')
transport.insert(1, '是否交通', 1)
all_com = pd.concat([indestry, transport,], ignore_index=True)[['名称', '是否交通',
                                                              '最新评级', '企业性质', '是否上市','一级分类', '二级分类']]
del indestry, transport

list_company_property = ['民营企业', '地方国有企业', '中央国有企业']
all_com['企业性质'] = all_com['企业性质'].map(lambda x:x if x in list_company_property else '其他')

# 违约信息读取+筛选第一次

list_re_col = ['发生日期','名称',]#'发行时主体评级'

re_of_de = pd.read_excel(path+'report of defaulted.xlsx',)[:-2]
re_of_de = re_of_de[list_re_col]
gc.collect()
print('违约记录数:', re_of_de['名称'].shape[0])
re_of_de = re_of_de.groupby(['名称',], as_index=False).apply(lambda x:x.sort_values(['发生日期']).iloc[0])
print('违约公司数:', re_of_de['名称'].shape[0])
all_com['whetherin'] = all_com['名称'].isin(re_of_de['名称'])*1  ##  总表中的违约公司
print('总表里的违约公司数:', sum(all_com['whetherin']))
re_of_de['whetherin'] = re_of_de['名称'].isin(all_com['名称'])*1  ##   在总表中有记录的违约公司
print('缺失数:',re_of_de['名称'].shape[0]-sum(all_com['whetherin']))

loss_re_name = re_of_de.loc[re_of_de['whetherin']==0, '名称']
print('缺失的公司名如下:\n---------------------------\n', pd.unique(loss_re_name), re_of_de.shape)

re_of_de = re_of_de.loc[re_of_de['whetherin']==1,:]

re_of_de = re_of_de.iloc[:,:-1]
all_com = all_com.iloc[:,:-1]

违约记录数: 181
违约公司数: 74
总表里的违约公司数: 65
缺失数: 9
缺失的公司名如下:
---------------------------
 ['东兴金满堂商贸有限公司' '山东迪浩耐磨管道股份有限公司' '惠州侨兴电信工业有限公司' '惠州侨兴电讯工业有限公司'
 '甘肃华协农业生物科技股份有限公司' '甘肃宏良皮业股份有限公司' '百花医药集团股份有限公司' '鄂尔多斯市益通路桥有限公司'
 '陕西通海绒业股份有限公司'] (74, 3)


### 特殊筛选

In [6]:
n = 0
k = 0
drop_content = ['永泰能源股份有限公司', '上海华信国际集团有限公司']
drop_index = []
for i in re_of_de['名称']:
    if k == len(drop_content):
        break
    for j in drop_content:
        if j == i:
            drop_index.append(n)
            k += 1
    n += 1

re_of_de.drop(drop_index, inplace=True)

del n,k,drop_content,drop_index

# 时光机器-一个季度前

In [7]:
re_of_de['发生日期'] = re_of_de['发生日期'].map(lambda x:getxyb(x))

# 处理财报数据,如果要画图就在这里改

In [8]:
data_a = data_a.loc[data_a['名称'].isin(all_com['名称']),:]

re_of_de = re_of_de.merge(data_a, on=['名称',],)  #留下违约公司所有财报数据re_of_de
re_of_de = re_of_de.loc[re_of_de['发生日期'] >= re_of_de['报告期'],:]
re_of_de.dropna(thresh=re_of_de.shape[1]-10 ,inplace=True)
re_of_de.insert(0, 'target', (re_of_de['发生日期'].dt.year - re_of_de['报告期'].dt.year))

In [9]:
data_a.insert(0, 'target', -1)
fin_col = re_of_de.columns
data_a = data_a.loc[~data_a['名称'].isin(re_of_de['名称']), :]   #总的数据表中剔除re_of_de

data_2017 = data_a.loc[data_a['报告期'].dt.year == 2017, :]
data_a = data_a.loc[data_a['报告期'].dt.year != 2017, :]

data_a.dropna(thresh=data_a.shape[1]-6, inplace=True)  #总表剔除nan

drop_number = 2
drop_dict = {}
for k in range(drop_number):
    col_ = np.random.choice(data_a.columns[3:-1],len(data_a.columns[3:-1]),replace=False,)
    for j in col_:
        data_a = drop_out(data_a, j, model='gauss', alpha=3)

a = pd.concat([re_of_de, data_a, data_2017])[fin_col]

del re_of_de, data_a
gc.collect()

32

In [10]:
weight_dict = {}
a.iloc[:, 4:],weight_dict = diy_ss(a.iloc[:, 4:], weight_dict, 0)

### 特殊处理

In [11]:
data_ = a.iloc[-1:,:].copy()

for key in weight_dict:
    if key in data_.columns:
        if weight_dict[key] == 'log1p':
            data_.loc[:, key] = np.log1p(data_.loc[:, key])
        else :
            data_.loc[:, key] = (data_.loc[:, key] - weight_dict[key][0])/weight_dict[key][1]

a.iloc[-1:,:] = data_

aa = pd.read_excel('./y/2013-2018_y.xlsx')

a = pd.concat([aa.iloc[:-1,:-1],data_])

In [11]:
a = pd.concat([a, pd.get_dummies(a['报告期'].dt.year)], axis=1)

com_flist = ['名称', '是否交通', '一级分类', '企业性质', '是否上市']
a = a.merge(all_com[com_flist], on='名称')

com_flist_ = com_flist.copy()
for o in ['名称', '一级分类']:
    com_flist_.remove(o)

for i in com_flist_:
    a = one_hot_str(a, i)

drop_length = 1
drop_length += len(set(a['报告期'].dt.year))
for i in com_flist_:
    drop_length += len(set(all_com[i]))

cols_a = list(a)
cols_a.insert(len(cols_a), cols_a.pop(cols_a.index('一级分类')))
a = a[cols_a]

a.iloc[:, 4:-1] = fillna_(a.iloc[:, 4:-1],a['一级分类'],n_epoch=4)

新材料 0/19 速动比率 2/8
新材料 0/19 货币资金/总债务 3/8
房地产开发 1/19 总资产报酬率(%) 12/29
房地产开发 1/19 经营性现金流/EBITDA 25/29
房地产开发 1/19 EBITDA/营业总收入 27/29
商务服务业 2/19 带息债务(亿元) 0/16
商务服务业 2/19 总资产报酬率(%) 3/16
商务服务业 2/19 流动比率 4/16
交通运输 3/19 筹资活动现金流(亿元) 10/29
交通运输 3/19 资产负债率 15/29
交通运输 3/19 带息债务/总投入资本 17/29
交通运输 3/19 货币资金/短期债务 18/29
交通运输 3/19 主营业务收入增长率(%) 22/29
交通运输 3/19 EBITDA(亿元) 23/29
交通运输 3/19 EBITDA/带息债务 26/29
高端装备 4/19 总资产(亿元) 0/27
高端装备 4/19 带息债务(亿元) 4/27
高端装备 4/19 净债务(亿元) 5/27
高端装备 4/19 筹资活动现金流(亿元) 8/27
高端装备 4/19 主营业务利润率(%) 11/27
高端装备 4/19 货币资金/总债务 21/27
物流快递 5/19 主营业务收入增长率(%) 3/9
物流快递 5/19 存货周转率 4/9
物流快递 5/19 EBITDA(亿元) 5/9
物流快递 5/19 EBITDA/营业总收入 8/9
必需品 6/19 总资产(亿元) 0/28
必需品 6/19 主营业务利润(亿元) 8/28
必需品 6/19 净资产回报率(%) 12/28
必需品 6/19 流动比率 13/28
必需品 6/19 经营性现金流/EBITDA 25/28
必需品 6/19 EBITDA/带息债务 27/28
节能环保 7/19 货币资产(亿元) 1/28
节能环保 7/19 净资产(亿元) 2/28
节能环保 7/19 总债务(亿元) 3/28
节能环保 7/19 经营活动现金流(亿元) 6/28
节能环保 7/19 投资活动现金流(亿元) 7/28
节能环保 7/19 主营业务利润(亿元) 8/28
节能环保 7/19 主营业务利润率(%) 10/28
节能环保 7/19 总资产报酬率(%) 11/28
节能环保 7/19 流动比率

In [12]:
def rechange(dataframe, col, weight):
    if col in weight.keys() :
        if weight[col] == 'log1p':
            return np.exp(dataframe[col]) - 1
        else:
            return dataframe[col] * weight[col][1] + weight[col][0]
    else :
        return dataframe[col]

a = a.iloc[:, :drop_length * -1]
a.insert(a.shape[1],'净利润/带息债务',rechange(a,'净利润(亿元)',weight_dict)/((rechange(a,'带息债务(亿元)',weight_dict))+0.01))
a[['净利润/带息债务']],weight_dict = diy_ss(a[['净利润/带息债务']],weight_dict,0)

In [16]:
# a.to_excel('./2013-2018_y.xlsx',index=False)
# joblib.dump(weight_dict,'./weight_dict_y.m')

In [ ]:
fin_col = re_of_de.columns

del re_of_de
gc.collect()

a = pd.read_excel('./2013-2018_y.xlsx')
weight_dict = joblib.load('./weight_dict_y.m')

# 违约范围划定

In [13]:
a.loc[a['发生日期'].notnull(),'target'] = a.loc[a['发生日期'].notnull(),'target'].map(lambda x: 1 if x == 0 else 2)

In [14]:
for com_name in ['永泰能源股份有限公司', '上海华信国际集团有限公司']:
    a.loc[a['名称']==com_name,'target'] = -1

In [15]:
a.drop('发生日期',axis=1,inplace=True)
re_of_de_0 = a.loc[a['target']==2,:]
re_of_de_1 = a.loc[a['target']==1,:]
data_2017 = a.loc[(a['报告期'].dt.year == 2017)&(a['target'] == -1),:]
data_a = a.loc[(a['报告期'].dt.year != 2017)&(a['target'] == -1),:]
del a
data_a = data_a.loc[data_a['报告期'].between('2014-01-01','2018-03-31')]
gc.collect()

86

# 处理公司非财报数据

In [16]:
all_com = one_hot_str(all_com,'企业性质',replace=False)
all_com = one_hot_str(all_com,'是否上市',replace=False)

re_of_de_0 = re_of_de_0.merge(all_com,on='名称',)
re_of_de_1 = re_of_de_1.merge(all_com,on='名称',)
data_2017 = data_2017.merge(all_com,on='名称',)
data_a = data_a.merge(all_com,on='名称')
del all_com

# 看看corr

In [21]:
a = [('净资产回报率(%)', 0.2831613389507573),
 ('净利润(亿元)', 0.19612521767114766),
 ('流动比率', 0.10006272874771727),
 ('净资产(亿元)', 0.07643978218479412),
 ('主营业务利润(亿元)', 0.06485263851265907),
 ('货币资金/短期债务', 0.04648683392338189),
 ('总资产报酬率(%)', 0.03358045966497278),
 ('筹资活动现金流(亿元)', 0.0299973498895215),
 ('投资活动现金流(亿元)', 0.02486080556388141),
 ('总资产(亿元)', 0.02458365655281366),
 ('短期债务/总债务', 0.023363955568147503),
 ('经营活动现金流(亿元)', 0.01991941294740471),
 ('速动比率', 0.017362472591467188),
 ('货币资产(亿元)', 0.013813465657292835),
 ('带息债务(亿元)', 0.012756002097363587),
 ('主营业务收入(亿元)', 0.00853891335415696),
 ('净债务(亿元)', 0.008103904697081556),
 ('主营业务利润率(%)', 0.005525472849658592),
 ('资产负债率', 0.0040236587366119),
 ('存货周转率', 0.002683313003464182),
 ('总债务(亿元)', 0.002371458968943401),]
pd.concat([re_of_de,data_2018,data_a])[[i[0] for i in a]].corr().to_excel(path+'corr.xlsx',)

In [13]:
# list_choich = ['名称', '报告期','获息倍数','货币资金/短期债务','净利润亿元','筹资活动现金流亿元','主营业务利润率%',
#                 '主营业务收入增长率%','投资活动现金流亿元','净资产回报率%','总资产报酬率%','流动比率']


In [14]:
# for i in [re_of_de,data_2018]:
#     i.fillna(0,inplace=True)

# 把标签特征移入data_object

In [17]:
object_list = ['名称', '报告期','最新评级','企业性质', '是否上市', '一级分类', '二级分类',]
data_a_object = pd.DataFrame();data_2017_object = pd.DataFrame();re_object_0 = pd.DataFrame();re_object_1 = pd.DataFrame()

for i in object_list:
    data_a_object = pd.concat([data_a_object,data_a.pop(i)],axis=1)
for i in object_list:
    data_2017_object = pd.concat([data_2017_object,data_2017.pop(i)],axis=1)
for i in object_list:
    re_object_0 = pd.concat([re_object_0,re_of_de_0.pop(i)],axis=1)
for i in object_list:
    re_object_1 = pd.concat([re_object_1,re_of_de_1.pop(i)],axis=1)

In [18]:
re_target_0 = re_of_de_0.pop('target')
re_target_0 = re_target_0.map(lambda x : 0)
re_target_1 = re_of_de_1.pop('target')
data_a_target = data_a.pop('target')
data_a_target = data_a_target.map(lambda x : 0)
data_2017.drop(['target'],axis=1,inplace=True)

# 数据集 -> x1,x2,y1,y2

In [19]:
def get_hxy(length_0=0.6,length_a=0.022,test_size=0.3,simple=False,random_state=None):
    '''横向数据'''
    global x1,x2,y1,y2,list_
    list_ = re_of_de_1.columns
    index_a = np.random.permutation(len(data_a_target))[:int(len(data_a_target)*length_a)]
    x1,x2,y1,y2 = train_test_split(pd.concat([re_of_de_1,data_a.iloc[index_a,:]],),
                                   np.r_[re_target_1,[0]*(len(index_a))],test_size=test_size,random_state=random_state)
    if simple:
        pass

def get_zxy(length_0=0.6,test_size=0.3,simple=False,random_state=None):
    '''纵向数据'''
    global x1,x2,y1,y2,list_
    list_ = re_of_de_1.columns
    index_0 = np.random.permutation(len(re_target_0))[:int(len(re_target_0)*length_0)]
    x1,x2,y1,y2 = train_test_split(pd.concat([re_of_de_1,re_of_de_0.iloc[index_0,:],],),
                                   np.r_[re_target_1,[0]*(len(index_0))],test_size=test_size,random_state=random_state)
    if simple:
#         list_ = ['获息倍数','货币资金/短期债务','净利润(亿元)','筹资活动现金流(亿元)','主营业务利润率(%)',
#                  '主营业务收入增长率(%)','投资活动现金流(亿元)','净资产回报率(%)','总资产报酬率(%)','流动比率',]
        list_ = ['获息倍数','货币资金/短期债务','净利润(亿元)','筹资活动现金流(亿元)','主营业务利润率(%)','投资活动现金流(亿元)',
       '主营业务收入增长率(%)','经营活动现金流(亿元)','净资产回报率(%)','总资产报酬率(%)','EBITDA/带息债务','流动比率',]
        x1 = x1[list_]
        x2 = x2[list_]

# hpsklearn

In [20]:
from hyperopt import tpe,fmin,hp
from hpsklearn import random_forest,HyperoptEstimator

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [21]:
def model_score(args):
    n_estimators ,max_depth ,min_samples_split ,max_features = args
    model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,
                                   class_weight='balanced',n_jobs=-1,
                                   min_samples_split=min_samples_split,max_features=max_features,)
    model.fit(x1,y1)
    pre = model.predict(x2)
    recall_ = metrics.recall_score(pre,y2)
    accu_ = metrics.accuracy_score(pre,y2)
    auc_ = metrics.roc_auc_score(pre,y2)
    score_ = -(recall_*0.4+auc_*0.3+accu_*0.3)
    return score_

space_dict = {
              'n_estimators':[i for i in range(30,121,5)],
              'max_depth':[2,3,4,5,],
              'min_samples_split':[i for i in range(2,11,2)],
              'max_features':[i/100 for i in range(50,71,5)],
              }
space = [hp.choice(i,space_dict[i]) for i in space_dict.keys()]

from functools import reduce
result_list = []
for i in range(1):
    get_zxy(length_0=1,simple=True,random_state=i*10+5)
#     index = fmin(model_score,space,tpe.suggest,300)
    index = fmin(model_score,space,tpe.suggest,reduce(lambda x,y:x*y,[len(space_dict[i]) for i in space_dict]))

    result_dict = {}
    for i in index.keys():
        result_dict[i] = space_dict[i][index[i]]
    rfc = RandomForestClassifier(n_estimators = result_dict['n_estimators'],
                                 max_depth = result_dict['max_depth'],
                                 min_samples_split = result_dict['min_samples_split'],
                                 max_features = result_dict['max_features'],
                                 class_weight = 'balanced', n_jobs = -1,warm_start=True,)
    result_list.append((rfc,result_dict))

In [24]:
for n,(i,_) in enumerate(result_list):
    get_zxy(length_0=1,simple=True,random_state=n*10)
    i.fit(x1,y1)
# metrics.recall_score(y2,rfc.predict(x2))
    print(0.3*metrics.recall_score(y2,i.predict(x2))+\
          0.4*metrics.roc_auc_score(y2,i.predict(x2))+\
          0.3*metrics.accuracy_score(y2,i.predict(x2)))

0.4913105413105413


In [25]:
for j in zip(['data_a','re_of_de_0','re_of_de_1'],['data_a_target','re_target_0','re_target_1']):
    pre = [i[0].predict_proba(eval(j[0])[list_])[:,1] for i in result_list]
    pre = [sum(i)/len(result_list) for i in zip(*pre)]
    pre = [1 if i>=0.5 else 0 for i in pre]
    print(f'---{j[0]}---\n',metrics.confusion_matrix(eval(j[1]),pre),'\n-----------')

---data_a---
 [[5198  169]
 [   0    0]] 
-----------
---re_of_de_0---
 [[95  2]
 [ 0  0]] 
-----------
---re_of_de_1---
 [[ 0  0]
 [11 20]] 
-----------


In [38]:
for n,(i,_) in enumerate(result_list):
    if n == 0:
        feature_importances = pd.Series(i.feature_importances_)
    else:
        feature_importances += pd.Series(i.feature_importances_)
feature_importances = pd.DataFrame(feature_importances/feature_importances.sum(),columns=['feature_importance'],)
feature_importances['id'] = list_
feature_importances.set_index(['id'],inplace=True)
feature_importances.sort_values(by=['feature_importance'],ascending=False).iloc[:10,:]

,feature_importance
id,
净资产回报率(%),0.206418
筹资活动现金流(亿元),0.162879
总资产报酬率(%),0.142255
净利润(亿元),0.103224
经营活动现金流(亿元),0.071721
主营业务收入增长率(%),0.071298
EBITDA/带息债务,0.070777
货币资金/短期债务,0.065535
主营业务利润率(%),0.031029


In [39]:
data_a[list_].describe()

,获息倍数,货币资金/短期债务,净利润(亿元),筹资活动现金流(亿元),主营业务利润率(%),投资活动现金流(亿元),主营业务收入增长率(%),经营活动现金流(亿元),净资产回报率(%),总资产报酬率(%),EBITDA/带息债务,流动比率
count,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000
mean,8.488975,0.389735,-0.070506,-0.078119,0.391300,0.085012,18.743047,-0.050263,4.764667,4.536321,87.478779,1.484276
std,24.593617,0.258278,0.072538,0.177198,272.612518,0.084117,167.180947,0.076786,27.367407,4.471382,1044.264564,0.801250
min,-750.678500,0.000000,-1.344136,-0.892281,-14493.493000,-0.564672,-100.000000,-0.604623,-1510.377900,-61.845500,-2757.134923,0.033500
25%,1.854250,0.206644,-0.100254,-0.167654,0.786300,0.062229,-4.939100,-0.078460,1.881800,2.180400,10.453715,0.951700
50%,3.560518,0.328765,-0.081928,-0.117989,5.126300,0.110751,6.947300,-0.057321,5.102500,4.035200,18.173937,1.309300
75%,7.519434,0.511731,-0.044722,-0.016237,11.828850,0.134866,22.236350,-0.016928,10.402800,6.748300,31.384407,1.819800
max,584.759868,1.504838,0.191743,0.767130,610.352100,0.352443,7391.457886,0.213779,61.885200,19.200200,27652.289304,4.816800


In [26]:
pre_2018 = [i[0].predict_proba(data_2017[list_])[:,1] for i in result_list]
pre_2018 = [sum(i)/len(result_list) for i in zip(*pre_2018)]
pre_2018_ = [1 if i>=0.5 else 0 for i in pre_2018]

sum(pre_2018_)

135

In [27]:
joblib.dump(result_list[0],'./剔除华信永泰.m')

['./剔除华信永泰.m']

In [30]:
# data_2018_object.insert(0,'0-1 score',pre_2018)

In [31]:
data_2018_object.loc[[True if i==1 else False for i in pre_2018_],:].sort_values(by=['0-1 score'],ascending=False)

,0-1 score,名称,报告期,最新评级,企业性质,是否上市,一级分类,二级分类
1865,0.895059,山东龙力生物科技股份有限公司,2018-06-30,AA-,民营企业,是,必需品,食品
2571,0.868933,山东地矿股份有限公司,2018-06-30,AA-,地方国有企业,是,投资贸易,投资管理
1136,0.827068,华天酒店集团股份有限公司,2018-06-30,AA-,地方国有企业,是,可选消费品,酒店餐饮
511,0.759319,内蒙古矿业(集团)有限责任公司,2018-06-30,AA,地方国有企业,否,原材料采掘加工,有色
1704,0.737303,焦作万方铝业股份有限公司,2018-06-30,AA,其他,是,原材料采掘加工,有色
1445,0.729218,上海外滩投资开发(集团)有限公司,2018-06-30,AA+,地方国有企业,否,房地产开发,房地产
2128,0.726486,大唐电信科技股份有限公司,2018-06-30,AA-,中央国有企业,是,高端装备,集成电路
780,0.714669,暴风集团股份有限公司,2018-06-30,A+,民营企业,是,信息技术,软件开发
1556,0.712252,保定天威保变电气股份有限公司,2018-06-30,A+,中央国有企业,是,传统制造业,电气
2198,0.706287,哈尔滨工业投资集团有限公司,2018-06-30,AA,地方国有企业,否,投资贸易,投资管理


In [47]:
# joblib.dump(result_list,path+'../../m_save/2018-06-30_result list_a.m')

['./data/Debt issuing company 2018 report/../../m_save/2018-06-30_result list_a.m']

In [25]:
result_list = joblib.load(path+'../../m_save/2018-06-30_result list_a.m')